In [86]:
eq = [
    "10*x1 - x2 + 2*x3 - 6",
    "-x1 + 11*x2 - x3 + 3*x4 - 25",
    "2*x1 - x2 + 10*x3 - x4 + 11",
    "3*x2 - x3 + 8*x4 - 15",
]


In [87]:

import copy
import sympy as sym
from pprint import pprint
class Gauss:
    def __init__(self, _val:list) -> None:
        self._val = []
        self.__log = {0:
                      self._val}
    def include (self, __input:list, iter:int):
        self._val = __input
        self.__log[iter]= __input
        
    def output(self):
        return self.__log
class iterate:
    def __init__(self, _eq: list) -> None:
        self.eq = []

        for i in range(len(_eq)):
            _eq[i] = sym.sympify(_eq[i])
            _eq[i] = sym.solve(_eq[i], sym.Symbol(f"x{i + 1}"))[0]

        for i in range(len(_eq) + 1):
            globals()[f"x{i+1}"] = sym.Symbol(f"x{i+1}")

        for i in range(len(_eq)):
            self.eq.append(sym.sympify(str(_eq[i])))

        self.x = {0: ["0"] * len(_eq)}

       
        
    def solve(self, n_iter: int) -> list:
        self.process = Gauss(self.x[0])

        for i in range(n_iter):
            curr = copy.deepcopy(self.x[i])
            for j in range(len(self.eq)):
                subs_dict = {
                    sym.Symbol(f"x{k+1}"): float(curr[k])
                    for k in range(len(curr))
                }

                curr[j] = (self.eq[j].subs(subs_dict).evalf())
            self.x[i+1] = curr
            self.process.include(curr, i+1)

        self.hasil = self.x[n_iter]
    def output(self):
        return self.eq, self.hasil
    def log(self ):
        return self.process.output()
    
class convert:
    def __init__(self, __eq:list) -> None:
        self.matriks, self.vec  = sym.linear_eq_to_matrix(__eq, sym.symbols(f'x1:{len(__eq)+1}'))
    def ouput(self):
        return self.matriks, self.vec
    def dominant(self):
        return self.matriks.is_strongly_diagonally_dominant

In [88]:
pprint(convert(eq).ouput())
pprint(convert(eq).dominant())

(Matrix([
[10, -1,  2,  0],
[-1, 11, -1,  3],
[ 2, -1, 10, -1],
[ 0,  3, -1,  8]]),
 Matrix([
[  6],
[ 25],
[-11],
[ 15]]))
True


In [89]:

matriks = iterate(eq)
matriks.solve(10)
eq, ans = matriks.output()
pprint(eq)
pprint(ans)
pprint(matriks.log())

[x2/10 - x3/5 + 3/5,
 x1/11 + x3/11 - 3*x4/11 + 25/11,
 -x1/5 + x2/10 + x4/10 - 11/10,
 -3*x2/8 + x3/8 + 15/8]
[0.999999999986812, 1.99999999985957, -0.999999999976391, 1.00000000005561]
{0: [],
 1: [0.600000000000000,
     2.32727272727273,
     -0.987272727272727,
     0.878863636363636],
 2: [1.03018181818182, 2.03693801652893, -1.01445619834711, 0.984341219008264],
 3: [1.00658504132231, 2.00355501690458, -1.00252738467318, 0.998350945576634],
 4: [1.00086097862509, 2.00029825065655, -1.00030727610170, 0.999849746491082],
 5: [1.00009128028599, 2.00002134224646, -1.00003114718345, 0.999988103259647],
 6: [1.00000836366134, 2.00000117333627, -1.00000274507268, 0.999999216864815],
 7: [1.00000066634816, 2.00000002460737, -1.00000020912241, 0.999999964631935],
 8: [1.00000004428522, 1.99999999466064, -1.00000001292779, 1.00000000038629],
 9: [1.00000000205162, 1.99999999890591, -1.00000000048110, 1.00000000035015],
 10: [0.999999999986812,
      1.99999999985957,
      -0.999999999976